# CSC421 Assignment 1 Agents #

This notebook is based on the supporting material for topics covered in **Chapter 2 - Intelligent Agents** from the book *Artificial Intelligence: A Modern Approach.* This notebook uses implementations from [agents.py](https://github.com/aimacode/aima-python/blob/master/agents.py) module. Let's start by importing everything from agents module.

In [17]:
from agents import *
from notebook import psource
from ipythonblocks import BlockGrid

### ENVIRONMENT - Park

A park is an example of an environment because our dog can perceive and act upon it. The <b>Environment</b> class is an abstract class, so we will have to create our own subclass from it before we can use it.

In [2]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class ParkOrig(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
            
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles
    


# QUESTION 1A (Minimum) 0.5 points 

Copy the code above and modify the provided park environment so that instead of Food there are two more "things" called Tree and StainedTree. Replace the eat action action with one called mark that checks to see if there is a Tree at the agent's location and if there is one it changes it to a StainedTree at the same location. You can add a thing to a particular location with self.add_thing(t, location). Also change the is_done() method to stop the simulation when there are no Trees left to stain. Also move all agents to location 0 when the simulation is done. In addition, add a method to Park called show() that prints all locations up to a max_location. Example output would be: 

```python
park.show(5)
park.run(1)
park.show(5) 

0:[]
1:[<BlindDog>]
2:[]
3:[]
4:[]
BlindDog decided to move down at location: 1
0:[]
1:[]
2:[<BlindDog>]
3:[]
4:[]
```

In [3]:
# ANSWER 
# YOUR MODIFIED Park CODE GOES HERE 

class Tree(Thing):
    pass

class StainedTree(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def show(self, maxx):
        for i in range(maxx):
            print(i,":",self.list_things_at(i))
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "mark":
            items = self.list_things_at(agent.location, tclass=Tree)
            if len(items) != 0:
                if agent.stain(items[0]): #Have the dog stain the first item
                    print('{} marked {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    stained_tree = StainedTree()
                    self.add_thing(stained_tree, items[0].location)
        
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        
        
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_clean_trees = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_clean_trees


### PROGRAM - BlindDog
Now that we have a <b>Park</b> Class, we re-implement our <b>BlindDog</b> to be able to move down and eat food or drink water only if it is present.


In [4]:
class BlindDogOrig(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

# Question 1B (Minimum) 0.5 points 

Now its time to re-implement a <b>program</b> module for our dog that likes to mark. A program controls how the dog acts upon its environment. Our program will be very simple, and is shown in the table below. Change the code above to reflect this new program in which if there is no tree or water then the Blind Dog moves down, if there is water the Blind Dog drinks it, and if there is a tree, the dogs marks it by making it a Stained tree. 

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Tree</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>mark</td>
       <td>move down</td>
   </tr>
        
</table>

In [5]:
# ANSWER 
# Your modified BlindDog code goes here 
class BlindDog(Agent):
    location = 1
 
    def __init__(self, program=None):
        self.alive = True
        self.program = program
        
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        
    def stain(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Tree):
            return True
        return False

Let's now run our simulation by creating a park with some food, water, and our dog.

In [6]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Tree):
            return 'mark'
    return 'move down'

park = Park()
dog = BlindDog(program)
tree1 = Tree()
tree2 = Tree()
water = Water()
park.add_thing(dog, 1)
park.add_thing(tree1, 5)
park.add_thing(water, 7)
park.add_thing(tree2, 9)


"""
This should be the final output after running the simulation for 
enough steps for all the trees to be stained 

0:[<BlindDog>]
1:[]
2:[]
3:[]
4:[]
5:[<StainedTree>]
6:[]
7:[]
8:[]
9:[<StainedTree>]
"""


park.show(10)
park.run(6)
park.show(10)
park.run(6)
park.show(10)

0 : []
1 : [<BlindDog>]
2 : []
3 : []
4 : []
5 : [<Tree>]
6 : []
7 : [<Water>]
8 : []
9 : [<Tree>]
BlindDog decided to move down at location: 1
BlindDog decided to move down at location: 2
BlindDog decided to move down at location: 3
BlindDog decided to move down at location: 4
BlindDog marked Tree at location: 5
BlindDog decided to move down at location: 5
0 : []
1 : []
2 : []
3 : []
4 : []
5 : [<StainedTree>]
6 : [<BlindDog>]
7 : [<Water>]
8 : []
9 : [<Tree>]
BlindDog decided to move down at location: 6
BlindDog decided to move down at location: 7
BlindDog decided to move down at location: 8
BlindDog marked Tree at location: 9
BlindDog decided to move down at location: 9
BlindDog decided to move down at location: 10
0 : []
1 : []
2 : []
3 : []
4 : []
5 : [<StainedTree>]
6 : []
7 : [<Water>]
8 : []
9 : [<StainedTree>]


Perfect! Note how the simulation stopped after the dog marked all trees, as we had defined before. 

Above, we learnt to implement an agent, its program, and an environment on which it acts. However, this was a very simple case. Let's try to add complexity to it by creating a 2-Dimensional environment!


## AGENTS IN A 2D ENVIRONMENT

For us to not read so many logs of what our dog did, we add a bit of graphics while making our Park 2D. To do so, we will need to make it a subclass of <b>GraphicEnvironment</b> instead of Environment. Parks implemented by subclassing <b>GraphicEnvironment</b> class adds these extra properties to it:

 - Our park is indexed in the 4th quadrant of the X-Y plane.
 - Every time we create a park subclassing <b>GraphicEnvironment</b>, we need to define the colors of all the things we plan to put into the park. The colors are defined in typical [<b>RGB digital 8-bit format</b>](https://en.wikipedia.org/wiki/RGB_color_model#Numeric_representations), common across the web.
 - Fences are added automatically to all parks so that our dog does not go outside the park's boundary - it just isn't safe for blind dogs to be outside the park by themselves! <b>GraphicEnvironment</b> provides `is_inbounds` function to check if our dog tries to leave the park.
 
First let us try to upgrade our 1-dimensional `Park` environment by just replacing its superclass by `GraphicEnvironment`. 

In [7]:
class Park2DOrig(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDogOrig(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

# QUESTION 1C (Expected) 0.5 points 

Modify the ParkDOrig and BlindDogOrig code above to support the mark action in two dimensions. **NOTE** The agent location is represented as a list so be careful about how you handle assignments which in Python by default sets 
two variables pointing to the same list. Change the isDone method so that the simulation stops when there are no trees left to mark and return agents to the start location. Also add a show method for printing the contents of the park - you can use self.width and self.height for the park dimensions. It is not required but it is a good idea to look at the contents of agents.py and understand how things are working under the hood. 

**NOTE** You will need to edit agents.py in a small way: 
Change the is_inbounds() method to: 


```python

    def is_inbounds(self, location):
        # Checks to make sure that the location is inbounds (within walls if we have walls)
        x, y = location
        return not (x < self.x_start or x >= self.x_end or y < self.y_start or y >= self.y_end)
```
This fixes a small bug that allowed the agent to be outiside the park at x_end or y_yend coordinates 
probably by mixing indexing from 0 and indexing from 1 

In [8]:
# ANSWER 
# Your modified code goes here 

class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "mark":
            items = self.list_things_at(agent.location, tclass=Tree)
            if len(items) != 0:
                if agent.mark(items[0]): #Have the dog staine the first item
                    print('{} marked {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    self.add_thing(StainedTree(), items[0].location)
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_things = not any(isinstance(thing, Food) or isinstance(thing, Water) or isinstance(thing, Tree) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_things

class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("right") # variable to store the direction our dog is facing
    
    def moveforward(self, success=True):
        if not success:
            return
        if self.direction.direction == Direction.R:
            if self.location[0] + 1 >= 20: return
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            if self.location[0] - 1 < 0: return
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            if self.location[1] < 0: return 
            self.location[1] -= 1
    
    def turn(self, d):
        print("CALLED")
        self.direction = self.direction + d
        print(d)
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
    def mark(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Tree):
            return True
        return False

Now let's test this new park with our same dog, food and water. We color our dog with a nice red and mark trees and water with orange and blue respectively. In addition, we color Stained trees yellow. 

In [9]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Tree):
            return 'mark'
        if isinstance(p, Food):
            return "eat"
        if isinstance(p, Water):
            return "drink"
        if isinstance(p, Bump):
            # pick left or right turn
            if random.choice((1,2)) == 1: return 'turnright'
            else: return 'turnleft'
            
    # If not at percept randomly turn or 
    # go forward
    choice = random.choice((1,2,3,4))
    choices = {
            1: 'turnright',
            2: 'turnleft',
            3: 'moveforward',
            4: 'moveforward'
    }
    print(choices[choice])
    return choices[choice]

park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'StainedTree':(255,255,0),'Tree': (230, 115, 40)}) # park width is set to 5, and height to 20
dog = BlindDog(program)
tree = Tree()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(tree, [0,5])
park.add_thing(water, [0,7])
tree2 = Tree()
park.add_thing(tree2, [0,9])

park.run(20)
#park.show()

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


turnright
BlindDog decided to turnright at location: [8, 8]
CALLED
right


KeyboardInterrupt: 

In [10]:
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


turnleft
BlindDog decided to turnleft at location: [7, 9]
CALLED
left


KeyboardInterrupt: 

Adding some graphics was a good idea! We immediately see that the code works, but our blind dog doesn't make any use of the 2 dimensional space available to him. Let's make our dog more energetic so that he turns and moves forward, instead of always moving down. In doing so, we'll also need to make some changes to our environment to be able to handle this extra motion.

### PROGRAM - EnergeticBlindDog

Let's make our dog turn or move forwards at random - except when he's at the edge of our park - in which case we make him change his direction explicitly by turning to avoid trying to leave the park. However, our dog is blind so he wouldn't know which way to turn - he'd just have to try arbitrarily.

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

# QUESTION 1D (Expected) 0.5 points 

In this question I ask you to add some logging to the park and change the BlindDog from a reflex-agent to a very simple model-based agent. First you will need to modify EnergeticBlindDog from the agents.ipynb notebook 
similarly to how we did above to support marking trees. For measuring the performance of our agent we can count the number of steps that are required to mark all the trees. As the agent behaves stochastically this number will vary. You can initialize the number of steps in the Park environment by adding an appropriate __init__ function. 
You will need to modify is_done appropriately so that it keeps track of the number steps and print the final number of steps when the simulation is completed. 

```Python
def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
     super().__init__(width, height, boundary, color, display)
     self.steps = 0 
        
```


The program of an Agent is a function that takes as input a list of percepts and 
returns an action. In order to create a model agent we have to maintain some state information. 
There is different ways to achieve this effect in Python. Here is an example 
of wrapping the program into a callable class i.e a class that can be called as a function. 
Here is an example of a Program that keeps track of whether the agent has drank water or not. 

Write a similar CountMarkingProgram that counts how many trees have been marked by the Agent. 
It is important to understand that the only way the agent can experience the environment 
is through the precepts. It would be easy to track the information about how many trees 
have been marked in the Environment and inform the agent BUT that would completely break 
the restriction that the Agent ONLY experiences the world through the precepts. 


```Python
class ThirstyProgram:

    def __init__(self): 
        self.thirsty = True 
    
    def program(self,percepts):
        '''Returns an action based on it's percepts'''
        print('Thirsty = ', self.thirsty)
        for p in percepts: # first eat or drink - you're a dog!
            if isinstance(p, Tree):
                return 'mark'
            elif isinstance(p, Water):
                self.thirsty = False 
                return 'drink'
            if isinstance(p,Bump): # then check if you are at an edge and have to turn
                turn = False
                choice = random.choice((1,2));
            else:
                choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
        if choice == 1:
            return 'turnright'
        elif choice == 2:
            return 'turnleft'
        else:
            return 'moveforward'
    
    def __call__(self, precepts): 
        return self.program(precepts)
```

In [14]:
# ANSWER 
class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    marked_trees = 0
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def mark(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Tree):
            self.marked_trees += 1
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

### ENVIRONMENT - Park2D

We also need to modify our park accordingly, in order to be able to handle all the new actions our dog wishes to execute. Additionally, we'll need to prevent our dog from moving to locations beyond our park boundary - it just isn't safe for blind dogs to be outside the park by themselves.

In [15]:
# ANSWER 
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "mark":
            items = self.list_things_at(agent.location, tclass=Tree)
            if len(items) != 0:
                if agent.mark(items[0]): #Have the dog staine the first item
                    print('{} marked {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    self.add_thing(StainedTree(), items[0].location)
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_things = not any(isinstance(thing, Tree) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_things


Now that our park is ready for the 2D motion of our energetic dog, lets test it!

In [16]:
park = Park2D(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Tree': (230, 115, 40),
             'StainedTree': (255, 255,0)})
dog = EnergeticBlindDog(program)
tree = Tree()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(tree, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
anothertree = Tree()
park.add_thing(morewater, [2,4])
park.add_thing(anothertree, [4,3])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")

# this should run until the simulation is done. The two trees should be stained and 
# the dog should be at [0,0]

showGUI = False
if (showGUI): 
    park.run()
else: 
    while not(park.is_done()): 
        park.step()

dog started at [0,0], facing down. Let's see if he found any food or water!
turnleft
EnergeticBlindDog decided to turnleft at location: [0, 0]
turnright
EnergeticBlindDog decided to turnright at location: [0, 0]
turnright
EnergeticBlindDog decided to turnright at location: [0, 0]
turnleft
EnergeticBlindDog decided to turnleft at location: [0, 0]
moveforward
EnergeticBlindDog decided to move downwards at location: [0, 0]
EnergeticBlindDog drank Water at location: [0, 1]
turnleft
EnergeticBlindDog decided to turnleft at location: [0, 1]
moveforward
EnergeticBlindDog decided to move rightwards at location: [0, 1]
turnright
EnergeticBlindDog decided to turnright at location: [1, 1]
moveforward
EnergeticBlindDog decided to move downwards at location: [1, 1]
EnergeticBlindDog marked Tree at location: [1, 2]
turnleft
EnergeticBlindDog decided to turnleft at location: [1, 2]
moveforward
EnergeticBlindDog decided to move rightwards at location: [1, 2]
turnleft
EnergeticBlindDog decided to turnl

turnright
EnergeticBlindDog decided to turnright at location: [-6, 36]
moveforward
EnergeticBlindDog decided to move leftwards at location: [-6, 36]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 36]
moveforward
EnergeticBlindDog decided to move upwards at location: [-7, 36]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 35]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 35]
moveforward
EnergeticBlindDog decided to move downwards at location: [-7, 35]
moveforward
EnergeticBlindDog decided to move downwards at location: [-7, 36]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 37]
turnleft
EnergeticBlindDog decided to turnleft at location: [-7, 37]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 37]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 37]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 37]
moveforward
EnergeticBlindDog decided to move rightwa

turnleft
EnergeticBlindDog decided to turnleft at location: [-7, 13]
moveforward
EnergeticBlindDog decided to move leftwards at location: [-7, 13]
turnright
EnergeticBlindDog decided to turnright at location: [-8, 13]
moveforward
EnergeticBlindDog decided to move upwards at location: [-8, 13]
moveforward
EnergeticBlindDog decided to move upwards at location: [-8, 12]
turnright
EnergeticBlindDog decided to turnright at location: [-8, 11]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-8, 11]
turnleft
EnergeticBlindDog decided to turnleft at location: [-7, 11]
moveforward
EnergeticBlindDog decided to move upwards at location: [-7, 11]
turnright
EnergeticBlindDog decided to turnright at location: [-7, 10]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-7, 10]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-6, 10]
turnright
EnergeticBlindDog decided to turnright at location: [-5, 10]
turnleft
EnergeticBlindDog decided 

EnergeticBlindDog decided to turnleft at location: [7, -25]
moveforward
EnergeticBlindDog decided to move downwards at location: [7, -25]
turnleft
EnergeticBlindDog decided to turnleft at location: [7, -24]
turnright
EnergeticBlindDog decided to turnright at location: [7, -24]
turnleft
EnergeticBlindDog decided to turnleft at location: [7, -24]
moveforward
EnergeticBlindDog decided to move rightwards at location: [7, -24]
moveforward
EnergeticBlindDog decided to move rightwards at location: [8, -24]
moveforward
EnergeticBlindDog decided to move rightwards at location: [9, -24]
moveforward
EnergeticBlindDog decided to move rightwards at location: [10, -24]
moveforward
EnergeticBlindDog decided to move rightwards at location: [11, -24]
turnleft
EnergeticBlindDog decided to turnleft at location: [12, -24]
turnright
EnergeticBlindDog decided to turnright at location: [12, -24]
turnright
EnergeticBlindDog decided to turnright at location: [12, -24]
moveforward
EnergeticBlindDog decided to m

EnergeticBlindDog decided to turnleft at location: [32, -36]
moveforward
EnergeticBlindDog decided to move upwards at location: [32, -36]
turnright
EnergeticBlindDog decided to turnright at location: [32, -37]
moveforward
EnergeticBlindDog decided to move rightwards at location: [32, -37]
moveforward
EnergeticBlindDog decided to move rightwards at location: [33, -37]
turnright
EnergeticBlindDog decided to turnright at location: [34, -37]
turnright
EnergeticBlindDog decided to turnright at location: [34, -37]
turnright
EnergeticBlindDog decided to turnright at location: [34, -37]
moveforward
EnergeticBlindDog decided to move upwards at location: [34, -37]
turnleft
EnergeticBlindDog decided to turnleft at location: [34, -38]
moveforward
EnergeticBlindDog decided to move leftwards at location: [34, -38]
moveforward
EnergeticBlindDog decided to move leftwards at location: [33, -38]
moveforward
EnergeticBlindDog decided to move leftwards at location: [32, -38]
turnright
EnergeticBlindDog de

...

moveforward
EnergeticBlindDog decided to move rightwards at location: [-190, 40]
turnleft
EnergeticBlindDog decided to turnleft at location: [-189, 40]
turnleft
EnergeticBlindDog decided to turnleft at location: [-189, 40]
turnleft
EnergeticBlindDog decided to turnleft at location: [-189, 40]
moveforward
EnergeticBlindDog decided to move downwards at location: [-189, 40]
turnright
EnergeticBlindDog decided to turnright at location: [-189, 41]
turnleft
EnergeticBlindDog decided to turnleft at location: [-189, 41]
moveforward
EnergeticBlindDog decided to move downwards at location: [-189, 41]
moveforward
EnergeticBlindDog decided to move downwards at location: [-189, 42]
moveforward
EnergeticBlindDog decided to move downwards at location: [-189, 43]
turnright
EnergeticBlindDog decided to turnright at location: [-189, 44]
moveforward
EnergeticBlindDog decided to move leftwards at location: [-189, 44]
moveforward
EnergeticBlindDog decided to move leftwards at location: [-190, 44]
turnright

turnright
EnergeticBlindDog decided to turnright at location: [-191, 49]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-191, 49]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-190, 49]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-189, 49]
turnleft
EnergeticBlindDog decided to turnleft at location: [-188, 49]
turnleft
EnergeticBlindDog decided to turnleft at location: [-188, 49]
turnright
EnergeticBlindDog decided to turnright at location: [-188, 49]
moveforward
EnergeticBlindDog decided to move upwards at location: [-188, 49]
moveforward
EnergeticBlindDog decided to move upwards at location: [-188, 48]
turnleft
EnergeticBlindDog decided to turnleft at location: [-188, 47]
moveforward
EnergeticBlindDog decided to move leftwards at location: [-188, 47]
turnright
EnergeticBlindDog decided to turnright at location: [-189, 47]
turnleft
EnergeticBlindDog decided to turnleft at location: [-189, 47]
moveforward
Energe

turnright
EnergeticBlindDog decided to turnright at location: [-155, 28]
turnleft
EnergeticBlindDog decided to turnleft at location: [-155, 28]
turnright
EnergeticBlindDog decided to turnright at location: [-155, 28]
turnleft
EnergeticBlindDog decided to turnleft at location: [-155, 28]
moveforward
EnergeticBlindDog decided to move upwards at location: [-155, 28]
moveforward
EnergeticBlindDog decided to move upwards at location: [-155, 27]
turnright
EnergeticBlindDog decided to turnright at location: [-155, 26]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-155, 26]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-154, 26]
turnright
EnergeticBlindDog decided to turnright at location: [-153, 26]
turnleft
EnergeticBlindDog decided to turnleft at location: [-153, 26]
turnleft
EnergeticBlindDog decided to turnleft at location: [-153, 26]
turnleft
EnergeticBlindDog decided to turnleft at location: [-153, 26]
moveforward
EnergeticBlindDog decid

EnergeticBlindDog decided to turnright at location: [-163, 27]
moveforward
EnergeticBlindDog decided to move leftwards at location: [-163, 27]
turnright
EnergeticBlindDog decided to turnright at location: [-164, 27]
turnleft
EnergeticBlindDog decided to turnleft at location: [-164, 27]
turnright
EnergeticBlindDog decided to turnright at location: [-164, 27]
moveforward
EnergeticBlindDog decided to move upwards at location: [-164, 27]
moveforward
EnergeticBlindDog decided to move upwards at location: [-164, 26]
turnleft
EnergeticBlindDog decided to turnleft at location: [-164, 25]
turnright
EnergeticBlindDog decided to turnright at location: [-164, 25]
moveforward
EnergeticBlindDog decided to move upwards at location: [-164, 25]
turnright
EnergeticBlindDog decided to turnright at location: [-164, 24]
moveforward
EnergeticBlindDog decided to move rightwards at location: [-164, 24]
turnright
EnergeticBlindDog decided to turnright at location: [-163, 24]
moveforward
EnergeticBlindDog decid

KeyboardInterrupt: 

In [ ]:


park = Park2D(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Tree': (230, 115, 40),
             'StainedTree': (255, 255,0)})
dog = EnergeticBlindDog(ThirstyProgram())
tree = Tree()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(tree, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
anothertree = Tree()
park.add_thing(morewater, [2,4])
park.add_thing(anothertree, [4,3])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.step()

In [ ]:
park.step()

In [ ]:
park.step()

In [ ]:
park.run()

# QUESTION 1E (Advanced)  - no points 

This question is not going to be graded but gives you some ideas of things to try if you want to explore agents further - the questions are much more open and you would have to refine them:  

1. Change the code so that if the dog drinks water it can only mark two trees. After that it can not mark until it drinks water again. 

2. Have the dog be able to sense the immediate neighbors i.e return any things that are located in the immediate neighborhood. 

3. Write a program that takes advantage of the sensing information. Show empirically by running multiple simulation runs that the dog that is able to sense performs better (on average finishes the simulation) faster 
than the energetic blind dog 

4. Add multiple dogs to the simulation 

5. Add agents that feed on other agents to model predator/prey interactions 

6. Write a goal-based agent version of the dog - with the explicit goal of marking every tree in the park 

7. Write a utility-based agent version of the dog - add complexity to the environment and multiple competing requirements 


